# Collect data comment Youtube Rewind Indonesia 2015 dengan Google Client API

Import library

In [1]:
from dotenv import load_dotenv
import os
import googleapiclient.discovery
import pandas as pd
import json
from pymongo import MongoClient

Mengakses value dari file env

In [2]:
# Memuat value dari file .env
load_dotenv()

dev = os.getenv('API_KEY_1')
mongodb_url = os.getenv('URL_SANDY')

Inisialisasi videoID dan database name

In [3]:
video_id = "fNX6Gu2F7A8"
database_name = "youtube_rewind_indonesia"
collection_name = "collect_2015"

Proses collect data

In [4]:
# Konfigurasi nama layanan API dan versi yang akan digunakan
api_service_name = "youtube"
api_version = "v3"

DEVELOPER_KEY = dev

# Koneksi ke MongoDB
client = MongoClient(mongodb_url)
db = client[database_name]
collection = db[collection_name]

# Membangun objek YouTube API dengan menggunakan kunci pengembang dan versi API yang ditentukan
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

# Membuat permintaan (request) untuk mendapatkan data komentar dari video tertentu
request = youtube.commentThreads().list(
    part="snippet",
    videoId=video_id,  # ID video yang akan diambil komentarnya
    maxResults=100  # Jumlah maksimum komentar yang akan diambil
)

# Inisialisasi list kosong untuk menyimpan komentar
comments = []

# Mengirimkan permintaan dan mendapatkan respons
response = request.execute()

counter = 1

# Mendapatkan komentar dari respons
for item in response['items']:
    comment = item['snippet']['topLevelComment']['snippet']
    public = item['snippet']['isPublic']
    # Menambahkan semua atribut komentar ke dalam list comments dalam bentuk dictionary
    comments.append({
        'authorDisplayName': comment['authorDisplayName'], # Nama penulis komentar
        'authorProfileImageUrl': comment['authorProfileImageUrl'], # URL gambar profil penulis
        'authorChannelUrl': comment['authorChannelUrl'], # URL channel penulis
        'channelId': comment['channelId'], # ID channel penulis
        'publishedAt': comment['publishedAt'], # Waktu publikasi komentar
        'updatedAt': comment['updatedAt'], # Waktu update komentar
        'likeCount': comment['likeCount'], # Jumlah suka komentar
        'textDisplay': comment['textDisplay'], # Isi komentar yang diformat dengan tautan dan emoji yang diproses
        'textOriginal': comment['textOriginal'], # Isi komentar
        'viewerRating': comment['viewerRating'], # Rating yang diberikan oleh pengguna
        'isPublic': public # Status publikasi komentar
    })

    # Menyisipkan data ke MongoDB
    collection.insert_one(comment)
    print(f"{counter} : {comment['textOriginal']}")
    counter+=1

# Melakukan iterasi untuk mengambil komentar dari halaman berikutnya jika ada
while (1 == 1):
    try:
        nextPageToken = response['nextPageToken']  # Mengambil token untuk halaman berikutnya
    except KeyError:
        break  # Keluar dari loop jika tidak ada halaman berikutnya
    
    nextPageToken = response['nextPageToken']  # Mengambil token untuk halaman berikutnya
    
    # Membuat permintaan baru dengan menggunakan token halaman berikutnya
    nextRequest = youtube.commentThreads().list(
        part="snippet", 
        videoId=video_id,  # ID video (opsional jika tetap sama dengan sebelumnya)
        maxResults=100,  # Jumlah maksimum komentar per halaman
        pageToken=nextPageToken  # Token untuk halaman berikutnya
    )
    # Mengirimkan permintaan baru dan mendapatkan respons
    response = nextRequest.execute()

    # Mendapatkan komentar dari respons dan menambahkannya ke dalam list comments
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        public = item['snippet']['isPublic']
        # Menambahkan semua atribut komentar ke dalam list comments dalam bentuk dictionary
        comments.append({
            'authorDisplayName': comment['authorDisplayName'], # Nama penulis komentar
            'authorProfileImageUrl': comment['authorProfileImageUrl'], # URL gambar profil penulis
            'authorChannelUrl': comment['authorChannelUrl'], # URL channel penulis
            'channelId': comment['channelId'], # ID channel penulis
            'publishedAt': comment['publishedAt'], # Waktu publikasi komentar
            'updatedAt': comment['updatedAt'], # Waktu update komentar
            'likeCount': comment['likeCount'], # Jumlah suka komentar
            'textDisplay': comment['textDisplay'], # Isi komentar yang diformat dengan tautan dan emoji yang diproses
            'textOriginal': comment['textOriginal'], # Isi komentar
            'viewerRating': comment['viewerRating'], # Rating yang diberikan oleh pengguna
            'isPublic': public # Status publikasi komentar
        })
        
        # Menyisipkan data ke MongoDB
        collection.insert_one(comment)
        print(f"{counter} : {comment['textOriginal']}")
        counter+=1

1 : Masih the best di hati oe
2 : 2024🎉
3 : 3:34 scene yg gw suka
4 : Diantara semua YouTube rewind, tahun ini yg paling favorit 😊😊
Bahkan tahun 2024 masih nonton, btw umur saya 22 tahun anak gen z yg rindu youtube rewind lama 😂
5 : ada yang balik kesini di 2024 ini rewind terbagus no debat❤😢
6 : Masih ada bryan disana
7 : ketika menonton rewind ini di masa saat ini(2024) serasa jadi time travelers, fakta memang dunia selalu berubah dan gak ketebak bakal menjadi apa, bersukur msh di kasih umur sampai saat ini😇🔥🔥🔥
8 : sumpah ini menurut gue rewind terbaik sih
9 : 2024 masih hadir
10 : era youtube terbaik, keluar dikit nih air mata kalo inget jaman itu, tiap hari nungguin salah satunya video arap
11 : Nge rewatch dan baru sadar ada rich brian 🗿
12 : 2024 dan masih mampir kesini, sehat-sehat ya kalian. tanpa sadar kalian telah menginspirasi kreator yang sekarang banyak di platform ini
13 : May 2024, and still watch this
14 : 2024 sni
15 : 3:22
16 : Dulu youtube rewind

Sekarang sirkel rew

KeyboardInterrupt: 

In [ ]:
# Membuat DataFrame dari list comments dengan kolom-kolom yang ditentukan
df = pd.DataFrame(comments)
# Menampilkan informasi DataFrame seperti jumlah baris dan tipe data kolom
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42100 entries, 0 to 42099
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   authorDisplayName      42100 non-null  object
 1   authorProfileImageUrl  42100 non-null  object
 2   authorChannelUrl       42100 non-null  object
 3   channelId              42100 non-null  object
 4   publishedAt            42100 non-null  object
 5   updatedAt              42100 non-null  object
 6   likeCount              42100 non-null  int64 
 7   textDisplay            42100 non-null  object
 8   textOriginal           42100 non-null  object
 9   viewerRating           42100 non-null  object
 10  isPublic               42100 non-null  bool  
dtypes: bool(1), int64(1), object(9)
memory usage: 3.3+ MB


In [ ]:
df

,authorDisplayName,authorProfileImageUrl,authorChannelUrl,channelId,publishedAt,updatedAt,likeCount,textDisplay,textOriginal,viewerRating,isPublic
0,@Tim2one,https://yt3.ggpht.com/ytc/AIdro_koWWb9JTasq6o_...,http://www.youtube.com/@Tim2one,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-12T08:40:31Z,2024-01-12T08:40:31Z,29683,Selamat menikmati REWIND INDONESIA di jam 19.4...,Selamat menikmati REWIND INDONESIA di jam 19.4...,none,True
1,@yanuaranandadwicahyo6395,https://yt3.ggpht.com/ytc/AIdro_kXPpC91hWBfq9k...,http://www.youtube.com/@yanuaranandadwicahyo6395,UC-z1ubmS-iHMVr83EEwRS2g,2024-05-23T20:59:13Z,2024-05-23T20:59:13Z,0,Tan boy ku gw kira maput jir,Tan boy ku gw kira maput jir,none,True
2,@rapratama__,https://yt3.ggpht.com/X8vCgNFgQO4aOiRQANFH9gZT...,http://www.youtube.com/@rapratama__,UC-z1ubmS-iHMVr83EEwRS2g,2024-05-23T18:54:10Z,2024-05-23T18:54:10Z,0,kudet bener kalau yt rewind indo udah tayang 😂😂,kudet bener kalau yt rewind indo udah tayang 😂😂,none,True
3,@AllPlayGame8,https://yt3.ggpht.com/UCXGnEXmVPZbNLU7eP2rwTSI...,http://www.youtube.com/@AllPlayGame8,UC-z1ubmS-iHMVr83EEwRS2g,2024-05-23T16:35:19Z,2024-05-23T16:35:19Z,0,Kalau lihat momen briyan jadi vilain anak peja...,Kalau lihat momen briyan jadi vilain anak peja...,none,True
4,@jvr11656,https://yt3.ggpht.com/ytc/AIdro_nkHMdD8-iph0EM...,http://www.youtube.com/@jvr11656,UC-z1ubmS-iHMVr83EEwRS2g,2024-05-23T01:51:39Z,2024-05-23T01:51:39Z,0,Baru ngeh setelah nnton 2x ada anaknya bang Ma...,Baru ngeh setelah nnton 2x ada anaknya bang Ma...,none,True
...,...,...,...,...,...,...,...,...,...,...,...
42095,@manarulhuda_6942,https://yt3.ggpht.com/ytc/AIdro_leToIhB16Pf0jI...,http://www.youtube.com/@manarulhuda_6942,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-12T14:38:15Z,2024-01-12T14:38:15Z,0,Gokilll bangettt keren semuanya.......🔥🔥,Gokilll bangettt keren semuanya.......🔥🔥,none,True
42096,@iamburgezsto,https://yt3.ggpht.com/ytc/AIdro_k0BVkUBr4SI6w8...,http://www.youtube.com/@iamburgezsto,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-12T14:38:15Z,2024-01-12T14:38:15Z,0,Sayang bgt @warintilOfficial udh 10jt subscrib...,Sayang bgt @warintilOfficial udh 10jt subscrib...,none,True
42097,@adeendh,https://yt3.ggpht.com/JkvszEXAT0J5hzZmYnZiEADZ...,http://www.youtube.com/@adeendh,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-12T14:38:12Z,2024-01-12T14:38:12Z,0,ada kak iaa!!!! omgg,ada kak iaa!!!! omgg,none,True
42098,@ahmadfadhil20,https://yt3.ggpht.com/UIjQM6X0adEDjJA6j8myufNA...,http://www.youtube.com/@ahmadfadhil20,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-12T14:38:11Z,2024-01-12T14:38:11Z,1,"Tahun lalu bang Windah masuk jadi Kratos, tapi...","Tahun lalu bang Windah masuk jadi Kratos, tapi...",none,True


In [ ]:
df.sort_values(by='likeCount', ascending=False)[0:10]

,authorDisplayName,authorProfileImageUrl,authorChannelUrl,channelId,publishedAt,updatedAt,likeCount,textDisplay,textOriginal,viewerRating,isPublic
0,@Tim2one,https://yt3.ggpht.com/ytc/AIdro_koWWb9JTasq6o_...,http://www.youtube.com/@Tim2one,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-12T08:40:31Z,2024-01-12T08:40:31Z,29683,Selamat menikmati REWIND INDONESIA di jam 19.4...,Selamat menikmati REWIND INDONESIA di jam 19.4...,none,True
38982,@gustafsailendra,https://yt3.ggpht.com/ytc/AIdro_nA02QFK2ACmE0t...,http://www.youtube.com/@gustafsailendra,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-12T15:37:18Z,2024-01-12T15:37:18Z,2878,Terimakasih semua atas kerjasamanya. KALIAN SE...,Terimakasih semua atas kerjasamanya. KALIAN SE...,none,True
20891,@EkaGustiwanaOfficial,https://yt3.ggpht.com/ytc/AIdro_nw2Od1WzMK3Uby...,http://www.youtube.com/@EkaGustiwanaOfficial,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-13T09:43:46Z,2024-01-13T09:43:46Z,2827,❤ Setiap tahun rewind selalu naik level. 17 me...,❤ Setiap tahun rewind selalu naik level. 17 me...,none,True
18159,@Marinasea,https://yt3.ggpht.com/xm2Ze-v2SOY_4vJjkkET-k3w...,http://www.youtube.com/@Marinasea,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-13T13:31:24Z,2024-01-13T13:31:24Z,2160,KEREN BANGET😭 CONGRATS BUAT SEMUA YANG UDAH TE...,KEREN BANGET😭 CONGRATS BUAT SEMUA YANG UDAH TE...,none,True
37967,@putriarianiofficial,https://yt3.ggpht.com/etfKh0w4c62FiVhyNzaEJnV2...,http://www.youtube.com/@putriarianiofficial,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-12T16:00:46Z,2024-01-12T16:00:46Z,2021,Yesss!!! Terima kasih rewind Indonesia. Seneng...,Yesss!!! Terima kasih rewind Indonesia. Seneng...,none,True
5054,@kevinkat655,https://yt3.ggpht.com/ytc/AIdro_lD_OtCjYfslcXV...,http://www.youtube.com/@kevinkat655,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-16T04:52:09Z,2024-01-16T04:52:09Z,1635,POINT REWIND KALI INI MENGGAMBARKAN :<br>1. Ra...,POINT REWIND KALI INI MENGGAMBARKAN :\n1. Raky...,none,True
38234,@NessieJudge,https://yt3.ggpht.com/ytc/AIdro_mKhX_UMt80XsIz...,http://www.youtube.com/@NessieJudge,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-12T15:54:09Z,2024-01-12T15:54:09Z,1069,Another masterpiece!! ❤️‍🔥 Congrats team <a hr...,Another masterpiece!! ❤️‍🔥 Congrats team #Rewi...,none,True
2163,@alfasierra7454,https://yt3.ggpht.com/ytc/AIdro_lFrphCPcRuLzxc...,http://www.youtube.com/@alfasierra7454,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-19T09:31:55Z,2024-01-19T09:31:55Z,1048,Dari awal sampai akhir habis dibikin merinding...,Dari awal sampai akhir habis dibikin merinding...,none,True
4557,@agungwibowo3531,https://yt3.ggpht.com/ytc/AIdro_nv6JKfp9Fd5_IC...,http://www.youtube.com/@agungwibowo3531,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-16T10:13:52Z,2024-01-17T00:17:24Z,972,"Time stamp<br><a href=""https://www.youtube.com...",Time stamp\n0:05 Presented by Hotto\n0:09 Adeg...,none,True
2283,@Jifakhiyyah,https://yt3.ggpht.com/0ISq2FlxEzQJMLpymZjrVavz...,http://www.youtube.com/@Jifakhiyyah,UC-z1ubmS-iHMVr83EEwRS2g,2024-01-19T04:14:10Z,2024-01-19T04:14:10Z,831,17 menit yang benar-benar menyadarkan kita aka...,17 menit yang benar-benar menyadarkan kita aka...,none,True
